<a href="https://colab.research.google.com/github/ochiait/Bridge-to-Practice-Course/blob/master/llm_jp_3_1_1_8b_ja_structured_train_n95_rev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#00_data-load

In [ ]:
import json, math
import pandas as pd
from datasets import load_dataset

# --- データセットの指定 ---
DATASET_REPO = "morizon/TCGA_Reports_ja_structured-filtered"

# --- データロード ---
ds_all = load_dataset(DATASET_REPO)
raw_ds = ds_all["train"]

total_len = len(raw_ds)
train_size = math.floor(total_len * 0.95)
raw_ds_train = raw_ds.select(range(0, train_size))
raw_ds_eval = raw_ds.select(range(train_size, total_len))

README.md:   0%|          | 0.00/386 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/352k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

#01_inference

In [ ]:
# @title 推論条件(学習前後で共通化)
GEN_KWARGS = dict(
    max_new_tokens=512,
    do_sample=True,
    top_p=0.95,
    temperature=0.7,
    repetition_penalty=1.05,
)

In [ ]:
# @title LLM-jp-3.1 推論用のライブラリのインストール
!pip install torch>=2.3.0 transformers>=4.40.1 tokenizers>=0.19.1 accelerate>=0.29.3 flash-attn>=2.5.8

###LLM-jp-3.1 Pre-trained Models
####llm-jp/llm-jp-3.1-1.8b

In [ ]:
inference_model_1="llm-jp/llm-jp-3.1-1.8b"

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(inference_model_1)
model = AutoModelForCausalLM.from_pretrained(inference_model_1, device_map="auto", torch_dtype=None)

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00001.safetensors:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

In [ ]:
print(model.dtype)
print(next(model.parameters()).dtype)

torch.float32
torch.float32


In [ ]:
import json

# ====================================================
# ▼▼▼ ステップ1：プロンプト構築の準備（★ここを追加） ▼▼▼
# ====================================================
SCHEMA_STR = (
    '{ "臓器": "", "採取方法": "", "診断": "", "分化度": "", '
    '"病期": "", "腫瘍径": "", "浸潤範囲": "", "断端": "", "その他所見": "" }'
)

def build_prompt(text):
    return (
        "#指示\n"
        "次の文章を以下のJSON形式に構造化してください。\n"
        f"{SCHEMA_STR}\n\n"
        "#抽出対象\n"
        f"{text}"
    )

In [ ]:
# ====================================================
# ▼▼▼ ステップ2：推論して保存（★ここを修正） ▼▼▼
# ====================================================
results = []

# raw_ds_eval のすべてのサンプルに対してループを実行
for i, sample in enumerate(raw_ds_eval):
    # 1. データセットから元のテキストを取得
    original_question = sample["question_ja"]

    # 2. build_prompt関数を使ってプロンプトを構築（★追加）
    prompt_text = build_prompt(original_question)

    # 3. 構築したプロンプトをトークナイズ（★入力変数を変更）
    tokenized_input = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt").to(model.device)

    # 4. 推論を実行
    with torch.no_grad():
        output = model.generate(tokenized_input, **GEN_KWARGS)[0]

    # 5. 生成された部分だけをデコード
    input_token_len = len(tokenized_input[0])
    generated_tokens = output[input_token_len:]
    output_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    # 6. 結果をリストに保存
    #    "input"には元のテキストを保存して、何を入力したかわかるようにする
    results.append({
        "input": original_question,
        "model_output": output_text.strip()
    })

    print(f"--- サンプル {i} 完了 ---") # 進捗表示（任意）

# ====================================================
# ▼▼▼ ステップ3：ファイルに保存（ここは変更なし） ▼▼▼
# ====================================================
# --- ファイル名をモデル名から自動生成 ---
# 例: "llm-jp/llm-jp-3.1-1.8b" → "llm-jp-3.1-1.8b"
modelname_1 = inference_model_1.split("/")[-1]
output_filename_1 = f"eval_result_{modelname_1}.json"

# --- JSONファイルに保存 ---
with open(output_filename_1, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n✅ 推論結果を {output_filename_1} に保存しました。")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- サンプル 0 完了 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- サンプル 1 完了 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- サンプル 2 完了 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- サンプル 3 完了 ---
--- サンプル 4 完了 ---

✅ 推論結果を eval_result_llm-jp-3.1-1.8b.json に保存しました。


In [ ]:
import pandas as pd

# --- 推論結果を DataFrame に変換 ---
df_results = pd.DataFrame(results)

# --- 表示設定 ---
pd.set_option('display.max_colwidth', None)   # 長文を省略せず表示
pd.set_option('display.max_rows', 100)        # 表示行数を調整（必要なら増やす）
pd.set_option('display.width', None)          # 横幅を自動調整

# --- Colab上でスクロール付きHTML表示 ---
from IPython.display import display, HTML

# 高さ・幅を指定してスクロール可能に
scrollable_html = df_results.to_html(escape=False)
display(HTML(f'<div style="max-height:600px; overflow-y:auto; border:1px solid #ccc;">{scrollable_html}</div>'))

# --- 保存ファイル確認メッセージ ---
print(f"✅ 推論結果を {output_filename_1} に保存しました。")
print("👇 下の表で全文スクロール表示できます。必要に応じて df_results を Colab の Data タブで開いてもOK。")

,input,model_output
0,病理報告書（追加説明）：散在するMIB-1反応性細胞が認められる。最も増殖活性の高い領域において、ラベリングインデックスは5.0％と算出される。これは中程度の増殖活性を示しており、一般的に低悪性度の組織学的所見と整合する。 \n追加診断：オリゴアストロサイトーマ、オリゴデンドログリオーマ優勢。低悪性度（WHO病理学分類II度）。MIB-1ラベリングインデックス：5.0％。 \n顕微鏡的説明：切片は軽度から中程度の高細胞性を示す膠質腫瘍を示す。主な細胞群はマイクロジミストサイトである。細胞の少数のみが核周囲にハローを示す。また、核がやや長楕円形で、やや好酸性の細胞質を示す少数の細胞群も認められる。有糸分裂像は認められない。微小血管増殖や壊死は認められない。,"--- A: {\n ""臓器"": ""肝臓"",\n ""採取方法"": ""生検"",\n ""診断"": ""Mieb-1陽性肝細胞癌"",\n ""分化度"": ""低悪性度"",\n ""病期"": ""III度"",\n ""腫瘍径"": ""不明"",\n ""浸潤範囲"": ""不明"",\n ""断端"": ""不明"",\n ""その他所見"": ""散在するMIB-1反応性細胞が認められる、オリゴアストロサイトーマ、オリゴデンドログリオーマ優勢、低悪性度（WHO病理学分類II度）、MIB-1ラベリングインデックス：5.0％""\n}"
1,病理学的要約報告。右側腹部精巣の摘出；精巣胚芽腫；径6.5 cm；可用スライドに存在する精巣内に腫瘍が限定；血管内侵襲あり；精巣網への侵襲あり。＝精巣摘出術施行日：病理医。,＝精巣生検実施後。＝卵巣・子宮摘出術施行後。＝精巣癌確定診断。＝精巣癌ステージIV、リンパ節転移陽性、遠隔転移陰性、腹膜播種なし。＝精巣癌の切除と同時に、隣接部組織も全摘出。＝精巣切除時に精巣体側の精管は温存。＝精巣残存部分には精巣上体も存在。＝精巣残存部分から再発なし。＝精巣残存部分の一部に微小リンパ球細胞混じり。＝精巣残存部分より再発なし。＝精巣残存部分からの再発なく、手術不能部位なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝精巣残存部分より再発なし。＝
2,"性別：男性。人種：白人。報告日：最終病理診断。 \nA. 左側骨盤リンパ節；解剖：－20個のリンパ節、腫瘍なし（0/20）。 \nB. 右側骨盤リンパ節；解剖：－13個のリンパ節、腫瘍なし（0/13）。 \nC. 尿路膀胱、前立腺、精巣付属腺、精巣管；膀胱前立腺摘出術：－膀胱に浸潤性尿路上皮癌を伴い、局所的扁平上皮分化あり、病理学的パラメータを参照のこと。 \n－腫瘍は膀胱筋層に浸潤し、局所的顕微鏡的膀胱外軟部組織への拡散あり。 \n－前立腺尿道に局所的尿路上皮原位癌あり、前立腺管への拡散あり。 \n－尿道、尿管および軟部組織縁、腫瘍なし。 \nD. 右側遠位尿管；切除：－尿管の一部、腫瘍なし。 \nE. 双側精巣管；切除：－精巣管、腫瘍なし。 \n\n尿路上皮癌の病理学的パラメータ。 \n1. 腫瘍型：浸潤性尿路上皮癌、局所的扁平上皮分化あり（10%）。＜10% pen TSS 9/7/12.les。 \n2. 腫瘍のグレード：高グレード。 \n3. 浸潤深さ：顕微鏡的膀胱外軟部組織への拡散（スライド C8）。 \n4. 腫瘍の分布：単発；4.0 cm；右側側壁、前壁および後壁。 \n5. 尿管縁：腫瘍陰性。 \n6. 遠位尿道縁：腫瘍陰性。 \n7. 軟部組織縁または漿膜：腫瘍陰性。 \n8. リンパ節：腫瘍陰性（0/33）。 \n9. pTNM：pT3a,N0,MX。有効。チェックリストは、アメリカ共同腫瘍学会（AJCC）および国際がん連盟（UICC）の膀胱用第7版TNM分類システムを用いる。解釈は担当病理医により実施され、レジデント／フェローとレビューされた。 \n\n臨床歴：高グレード尿路上皮癌を有する60歳の男性が、膀胱前立腺摘出術およびイレオコンダクト術を施行中。 \n\n受領標本： \nA：左側骨盤リンパ節 \nB：右側骨盤リンパ節 \nC：膀胱、前立腺、精巣付属腺、精巣管 \nD：右側遠位尿管 \nE：双側精巣管 \n\n肉眼所見：標本は5つの容器に分けて受領され、それぞれ患者名および医療記録番号でラベル付けされている。 \nA. 最初の容器はさらに「左側骨盤リンパ節」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、淡黄色の組織塊で、6.0 x 4.5 x 2.0 cmの大きさ。22個のリンパ節が確認され、最大径は0.2～2.4 cmの範囲。以下の通り提出された： \nA1：最大のリンパ節を切断。 \nA2：1個のリンパ節を切断。 \nA3：1個のリンパ節を切断。 \nA4-5：各4個のリンパ節。 \nA6：3個のリンパ節。 \nA7-8：各4個のリンパ節。 \n\nB. 2番目の容器はさらに「右側骨盤リンパ節」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、淡黄色の組織塊で、10.5 x 8.5 x 2.0 cmの大きさ。15個のリンパ節が確認され、最大径は0.2～4.5 cmの範囲。以下の通り提出された： \nB1-2：最大のリンパ節。 \nB3：1個のリンパ節。 \nB4：1個のリンパ節を切断。 \nB5：1個のリンパ節。 \nB6-7：1個のリンパ節。 \nB8：3個のリンパ節。 \nB9：2個のリンパ節。 \nB10：5個のリンパ節。 \n\nC. 3番目の容器はさらに「膀胱、前立腺、精巣付属腺、精巣管」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、21 x 11 x 5.5 cmの膀胱前立腺摘出術標本で、9.5 x 8.5 x 5.0 cmの膀胱と付属脂肪組織、および4.2 x 3.8 x 3.0 cmの前立腺からなる。右側精巣付属腺は3.0 x 1.5 x 0.5 cm、右側精巣管は6.0 cm。左側精巣付属腺は2.8 x 1.5 x 0.6 cm、左側精巣管は6.5 cm。右側尿管 stump は長さ0.5 cm、直径0.9 cmまで拡張、左側は長さ0.6 cm、直径0.1 cm、両者とも完全で通気性のある腔を示す。右側前立腺は青インク、左側前立腺は黒インクで染色。膀胱は黒インクで染色。膀胱および前立腺は尿道に沿って前側に切開。前立腺は薄い、完全な膜様被膜に囲まれており、切断面ではゴム状、ピンク～淡黄色の尿道周囲の結節状変化を示し、離れた腫瘤や硬結は認めない。切開された膀胱は、右側側壁に4.0 x 3.5 x 1.0 cmの硬い、淡黄色の腫瘤を示し、後壁および前壁に拡がっている。切断面では、腫瘤は粘膜を貫通しており、深部縁から1.8 cmの位置にある。周囲の膀胱粘膜は浮腫、しわ状、ピンク～淡黄色で、壁厚は1.0～1.5 cm。三角部に隣接する両側尿管口が確認され、プローブで通気性あり。精巣付属脂肪組織内に2個のリンパ節候補が確認され、それぞれ0.4 cmおよび0.6 cm。代表的断面は以下の通り提出された： \nC1：遠位前立腺尿道縁（先端部）。 \nC2：右側尿管切除縁。 \nC3：左側尿管切除縁。 \nC4：前立腺先端部を連続的に切断。 \nC5-10：膀胱腫瘍、後側外側（C6-7、C9-10は連続）。 \nC11：膀胱頂部の腫瘍非関与粘膜。 \nC12：膀胱後壁の腫瘍非関与粘膜。 \nC13-14：膀胱腫瘍、前側外側。 \nC15：膀胱前壁の腫瘍非関与粘膜。 \nC16-18：先端部から基底部まで代表的右側前立腺。 \nC19-21：先端部から基底部まで代表的左側前立腺。 \nC22：2個のリンパ節候補。 \n\nD. 4番目の容器はさらに「右側遠位尿管」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、淡黄色の脂肪組織を伴う、方向不明の淡黄色組織片で、長さ2.5 cm、直径0.9 cm。標本はすべてD1-2に提出された。 \n\nE. 5番目の容器はさらに「双側精巣管」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、ピンク～淡黄色の組織片2個で、精巣管に一致する。1つは長さ0.5 cm、直径

✅ 推論結果を eval_result_llm-jp-3.1-1.8b.json に保存しました。
👇 下の表で全文スクロール表示できます。必要に応じて df_results を Colab の Data タブで開いてもOK。


###LLM-jp-3.1 Fine-tuned Models
####llm-jp/llm-jp-3.1-1.8b-instruct4
####llm-jp/llm-jp-3.1-1.8の推論を実行した場合は、かならずランタイムを接続解除して削除を実施し、以下のセルを実行ください。

In [ ]:
import json, math
import pandas as pd
from datasets import load_dataset

# --- データセットの指定 ---
DATASET_REPO = "morizon/TCGA_Reports_ja_structured-filtered"

# --- データロード ---
ds_all = load_dataset(DATASET_REPO)
raw_ds = ds_all["train"]

total_len = len(raw_ds)
train_size = math.floor(total_len * 0.95)
raw_ds_train = raw_ds.select(range(0, train_size))
raw_ds_eval = raw_ds.select(range(train_size, total_len))

In [ ]:
# @title 推論条件(学習前後で共通化)
GEN_KWARGS = dict(
    max_new_tokens=512,
    do_sample=True,
    top_p=0.95,
    temperature=0.7,
    repetition_penalty=1.05,
)

In [ ]:
# @title LLM-jp-3.1 推論用のライブラリのインストール
!pip install torch>=2.3.0 transformers>=4.40.1 tokenizers>=0.19.1 accelerate>=0.29.3 flash-attn>=2.5.8

In [ ]:
inference_model_2="llm-jp/llm-jp-3.1-1.8b-instruct4"

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(inference_model_2)
model = AutoModelForCausalLM.from_pretrained(inference_model_2, device_map="auto", torch_dtype=None)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
import json
import torch # torch.no_grad() のためにインポート

# ====================================================
# ▼▼▼ ステップ1：プロンプトの元となるスキーマを定義（変更なし） ▼▼▼
# ====================================================
SCHEMA_STR = (
    '{ "臓器": "", "採取方法": "", "診断": "", "分化度": "", '
    '"病期": "", "腫瘍径": "", "浸潤範囲": "", "断端": "", "その他所見": "" }'
)

# ====================================================
# ▼▼▼ ステップ2：推論して保存（★ここを全面的に修正） ▼▼▼
# ====================================================
results = []

# 【推奨】警告メッセージを抑制し、安定した推論を行うための設定
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# raw_ds_eval のすべてのサンプルに対してループを実行
for i, sample in enumerate(raw_ds_eval):
    # 1. データセットから元のテキストを取得
    original_question = sample["question_ja"]

    # 2. モデルに入力するための「対話形式のプロンプト」を作成
    #    元の build_prompt の内容を user ロールに集約します
    user_prompt = (
        "#指示\n"
        "次の文章を以下のJSON形式に構造化してください。\n"
        f"{SCHEMA_STR}\n\n"
        "#抽出対象\n"
        f"{original_question}"
    )

    chat = [
        # systemロールはモデルの役割を定義するのに役立ちます（省略可能な場合もあります）
        {"role": "system", "content": ""},
        {"role": "user", "content": user_prompt}
    ]

    # 3. チャットテンプレートを適用してトークナイズ
    #    add_generation_prompt=True で、モデルが応答を始めるためのトリガー（例: <|assistant|>）を追加
    tokenized_input = tokenizer.apply_chat_template(
        chat,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt"
    ).to(model.device)

    # 4. 推論を実行
    with torch.no_grad():
        output = model.generate(
            tokenized_input,
            **GEN_KWARGS, # max_new_tokensなどの設定はGEN_KWARGSにまとめてあると仮定
        )[0]

    # 5. 生成された部分だけをデコード
    #    入力トークン（プロンプト全体）の長さを取得
    input_token_len = len(tokenized_input[0])
    # 生成された部分のトークンだけをスライスで取り出す
    generated_tokens = output[input_token_len:]
    # トークンをテキストにデコード
    output_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    # 6. 結果をリストに保存
    results.append({
        "input": original_question,
        "model_output": output_text.strip()
    })

    print(f"--- サンプル {i} 完了 ---")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- サンプル 0 完了 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- サンプル 1 完了 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- サンプル 2 完了 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--- サンプル 3 完了 ---
--- サンプル 4 完了 ---


In [ ]:
# ====================================================
# ▼▼▼ ステップ3：ファイルに保存（ここは変更なし） ▼▼▼
# ====================================================
# --- ファイル名をモデル名から自動生成 ---
modelname_2 = inference_model_2.split("/")[-1]
output_filename_2 = f"eval_result_{modelname_2}.json"

# --- JSONファイルに保存 ---
with open(output_filename_2, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n✅ 推論結果を {output_filename_2} に保存しました。")


✅ 推論結果を eval_result_llm-jp-3.1-1.8b-instruct4.json に保存しました。


In [ ]:
import pandas as pd

# --- 推論結果を DataFrame に変換 ---
df_results = pd.DataFrame(results)

# --- 表示設定 ---
pd.set_option('display.max_colwidth', None)   # 長文を省略せず表示
pd.set_option('display.max_rows', 100)        # 表示行数を調整（必要なら増やす）
pd.set_option('display.width', None)          # 横幅を自動調整

# --- Colab上でスクロール付きHTML表示 ---
from IPython.display import display, HTML

# 高さ・幅を指定してスクロール可能に
scrollable_html = df_results.to_html(escape=False)
display(HTML(f'<div style="max-height:600px; overflow-y:auto; border:1px solid #ccc;">{scrollable_html}</div>'))

,input,model_output
0,病理報告書（追加説明）：散在するMIB-1反応性細胞が認められる。最も増殖活性の高い領域において、ラベリングインデックスは5.0％と算出される。これは中程度の増殖活性を示しており、一般的に低悪性度の組織学的所見と整合する。 \n追加診断：オリゴアストロサイトーマ、オリゴデンドログリオーマ優勢。低悪性度（WHO病理学分類II度）。MIB-1ラベリングインデックス：5.0％。 \n顕微鏡的説明：切片は軽度から中程度の高細胞性を示す膠質腫瘍を示す。主な細胞群はマイクロジミストサイトである。細胞の少数のみが核周囲にハローを示す。また、核がやや長楕円形で、やや好酸性の細胞質を示す少数の細胞群も認められる。有糸分裂像は認められない。微小血管増殖や壊死は認められない。,"```json\n{\n ""臓器"": ""不明"", // 文章内では具体的な臓器名は記載されていません\n ""採取方法"": ""不明"", // 文章内では具体的な採取方法は記載されていません\n ""診断"": {\n ""主要所見"": ""散在するMIB-1反応性細胞"",\n ""追加所見"": ""オリゴアストロサイトーマ、オリゴデンドログリオーマ優勢。低悪性度（WHO病理学分類II度）""\n },\n ""分化度"": ""中程度"", // 明確な数値は記載されていませんが、MIB-1ラベリングインデックスにより推測されます\n ""病期"": ""低悪性度（WHO病理学分類II度）"",\n ""腫瘍径"": ""不明"", // 文章内には直接的なサイズ情報はありません\n ""浸潤範囲"": ""不明"", // 文章内には直接的な浸潤範囲の情報はありません\n ""断端"": ""不明"", // 文章内には直接的な断端に関する情報はありません\n ""その他所見"": ""軽度から中程度の高細胞性を示す膠質腫瘍。主な細胞群はマイクロジミストサイトであり、少数の細胞が核周囲にハローを示す。核がやや長楕円形で、やや好酸性の細胞質を示す少数の細胞群も認められる。有糸分裂像は認められず、微小血管増殖や壊死は認められない。""\n}\n```"
1,病理学的要約報告。右側腹部精巣の摘出；精巣胚芽腫；径6.5 cm；可用スライドに存在する精巣内に腫瘍が限定；血管内侵襲あり；精巣網への侵襲あり。＝精巣摘出術施行日：病理医。,"以下のJSON形式で、指定された情報を構造化しました。\n\n```json\n{\n ""臓器"": ""精巣"",\n ""採取方法"": ""精巣胚芽腫のため、精巣内での摘出術"",\n ""診断"": ""精巣胚芽腫"",\n ""分化度"": ""詳細な分化度は記載なし"",\n ""病期"": ""精巣胚芽腫"",\n ""腫瘍径"": ""6.5 cm"",\n ""浸潤範囲"": ""精巣網内"",\n ""断端"": ""詳細な記述なし"",\n ""その他所見"": ""血管内侵襲あり、精巣網への侵襲あり""\n}\n```\n\nただし、「断端」については、具体的な部位や状態についての情報が提供されていないため、上記のように空白としています。必要に応じて、追加情報がある場合にはその部分も含めて更新することができます。"
2,"性別：男性。人種：白人。報告日：最終病理診断。 \nA. 左側骨盤リンパ節；解剖：－20個のリンパ節、腫瘍なし（0/20）。 \nB. 右側骨盤リンパ節；解剖：－13個のリンパ節、腫瘍なし（0/13）。 \nC. 尿路膀胱、前立腺、精巣付属腺、精巣管；膀胱前立腺摘出術：－膀胱に浸潤性尿路上皮癌を伴い、局所的扁平上皮分化あり、病理学的パラメータを参照のこと。 \n－腫瘍は膀胱筋層に浸潤し、局所的顕微鏡的膀胱外軟部組織への拡散あり。 \n－前立腺尿道に局所的尿路上皮原位癌あり、前立腺管への拡散あり。 \n－尿道、尿管および軟部組織縁、腫瘍なし。 \nD. 右側遠位尿管；切除：－尿管の一部、腫瘍なし。 \nE. 双側精巣管；切除：－精巣管、腫瘍なし。 \n\n尿路上皮癌の病理学的パラメータ。 \n1. 腫瘍型：浸潤性尿路上皮癌、局所的扁平上皮分化あり（10%）。＜10% pen TSS 9/7/12.les。 \n2. 腫瘍のグレード：高グレード。 \n3. 浸潤深さ：顕微鏡的膀胱外軟部組織への拡散（スライド C8）。 \n4. 腫瘍の分布：単発；4.0 cm；右側側壁、前壁および後壁。 \n5. 尿管縁：腫瘍陰性。 \n6. 遠位尿道縁：腫瘍陰性。 \n7. 軟部組織縁または漿膜：腫瘍陰性。 \n8. リンパ節：腫瘍陰性（0/33）。 \n9. pTNM：pT3a,N0,MX。有効。チェックリストは、アメリカ共同腫瘍学会（AJCC）および国際がん連盟（UICC）の膀胱用第7版TNM分類システムを用いる。解釈は担当病理医により実施され、レジデント／フェローとレビューされた。 \n\n臨床歴：高グレード尿路上皮癌を有する60歳の男性が、膀胱前立腺摘出術およびイレオコンダクト術を施行中。 \n\n受領標本： \nA：左側骨盤リンパ節 \nB：右側骨盤リンパ節 \nC：膀胱、前立腺、精巣付属腺、精巣管 \nD：右側遠位尿管 \nE：双側精巣管 \n\n肉眼所見：標本は5つの容器に分けて受領され、それぞれ患者名および医療記録番号でラベル付けされている。 \nA. 最初の容器はさらに「左側骨盤リンパ節」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、淡黄色の組織塊で、6.0 x 4.5 x 2.0 cmの大きさ。22個のリンパ節が確認され、最大径は0.2～2.4 cmの範囲。以下の通り提出された： \nA1：最大のリンパ節を切断。 \nA2：1個のリンパ節を切断。 \nA3：1個のリンパ節を切断。 \nA4-5：各4個のリンパ節。 \nA6：3個のリンパ節。 \nA7-8：各4個のリンパ節。 \n\nB. 2番目の容器はさらに「右側骨盤リンパ節」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、淡黄色の組織塊で、10.5 x 8.5 x 2.0 cmの大きさ。15個のリンパ節が確認され、最大径は0.2～4.5 cmの範囲。以下の通り提出された： \nB1-2：最大のリンパ節。 \nB3：1個のリンパ節。 \nB4：1個のリンパ節を切断。 \nB5：1個のリンパ節。 \nB6-7：1個のリンパ節。 \nB8：3個のリンパ節。 \nB9：2個のリンパ節。 \nB10：5個のリンパ節。 \n\nC. 3番目の容器はさらに「膀胱、前立腺、精巣付属腺、精巣管」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、21 x 11 x 5.5 cmの膀胱前立腺摘出術標本で、9.5 x 8.5 x 5.0 cmの膀胱と付属脂肪組織、および4.2 x 3.8 x 3.0 cmの前立腺からなる。右側精巣付属腺は3.0 x 1.5 x 0.5 cm、右側精巣管は6.0 cm。左側精巣付属腺は2.8 x 1.5 x 0.6 cm、左側精巣管は6.5 cm。右側尿管 stump は長さ0.5 cm、直径0.9 cmまで拡張、左側は長さ0.6 cm、直径0.1 cm、両者とも完全で通気性のある腔を示す。右側前立腺は青インク、左側前立腺は黒インクで染色。膀胱は黒インクで染色。膀胱および前立腺は尿道に沿って前側に切開。前立腺は薄い、完全な膜様被膜に囲まれており、切断面ではゴム状、ピンク～淡黄色の尿道周囲の結節状変化を示し、離れた腫瘤や硬結は認めない。切開された膀胱は、右側側壁に4.0 x 3.5 x 1.0 cmの硬い、淡黄色の腫瘤を示し、後壁および前壁に拡がっている。切断面では、腫瘤は粘膜を貫通しており、深部縁から1.8 cmの位置にある。周囲の膀胱粘膜は浮腫、しわ状、ピンク～淡黄色で、壁厚は1.0～1.5 cm。三角部に隣接する両側尿管口が確認され、プローブで通気性あり。精巣付属脂肪組織内に2個のリンパ節候補が確認され、それぞれ0.4 cmおよび0.6 cm。代表的断面は以下の通り提出された： \nC1：遠位前立腺尿道縁（先端部）。 \nC2：右側尿管切除縁。 \nC3：左側尿管切除縁。 \nC4：前立腺先端部を連続的に切断。 \nC5-10：膀胱腫瘍、後側外側（C6-7、C9-10は連続）。 \nC11：膀胱頂部の腫瘍非関与粘膜。 \nC12：膀胱後壁の腫瘍非関与粘膜。 \nC13-14：膀胱腫瘍、前側外側。 \nC15：膀胱前壁の腫瘍非関与粘膜。 \nC16-18：先端部から基底部まで代表的右側前立腺。 \nC19-21：先端部から基底部まで代表的左側前立腺。 \nC22：2個のリンパ節候補。 \n\nD. 4番目の容器はさらに「右側遠位尿管」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、淡黄色の脂肪組織を伴う、方向不明の淡黄色組織片で、長さ2.5 cm、直径0.9 cm。標本はすべてD1-2に提出された。 \n\nE. 5番目の容器はさらに「双側精巣管」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、ピンク～淡黄色の組織片2個で、精巣管に一致する。1つは長さ0.5 cm、直径0.4 cm。もう1つは長さ4.5 cm、直径0.4 cm。代表的断面はE1に提出された。","```json\n{\n ""臓器"": ""尿路上皮"",\n ""採

#02_Finetune

以下のnotebookを参考に事後学習を行います  
https://github.com/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb

###unslothのライブラリーのインストール

In [ ]:
!pip install unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 5.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.3/351.3 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 945.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 111.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 36.4 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.23.0
    Uninstalling trl-0.23.0:
      Successfully uninstalled trl-0.23.0


In [ ]:
import json, math
import pandas as pd
from datasets import load_dataset

# --- データセットの指定 ---
DATASET_REPO = "morizon/TCGA_Reports_ja_structured-filtered"

# --- データロード ---
ds_all = load_dataset(DATASET_REPO)
raw_ds = ds_all["train"]

total_len = len(raw_ds)
train_size = math.floor(total_len * 0.95)
raw_ds_train = raw_ds.select(range(0, train_size))
raw_ds_eval = raw_ds.select(range(train_size, total_len))

###入力するtokenの上限設定および量子化の設定

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.9.0+cu130 with CUDA 1300 (you have 2.9.0+cu128)
    Python  3.10.19 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.9.0+cu130 with CUDA 1300 (you have 2.9.0+cu128)
    Python  3.10.19 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!


###モデルの指定およびロード

In [ ]:
MODEL_ID = "llm-jp/llm-jp-3.1-1.8b"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00001.safetensors:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

In [ ]:
print(model.config.name_or_path)
print(model.config._name_or_path)  # HFのキャッシュ名を明示
print(model.config.model_type)
print(model.config.architectures)
print(model.config.torch_dtype)
print(model.generation_config)

`torch_dtype` is deprecated! Use `dtype` instead!


llm-jp/llm-jp-3.1-1.8b
llm-jp/llm-jp-3.1-1.8b
llama
['LlamaForCausalLM']
torch.float16
GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 4
}



###学習時のloraの設定

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.2 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [ ]:
# from huggingface_hub import login
# from google.colab import userdata

# HF_TOKEN = userdata.get("HF_TOKEN")
# login(token=HF_TOKEN)

###学習データの整形

In [ ]:
# === ③ 出力スキーマ（固定形式） ===
SCHEMA_STR = (
    '{{ "臓器": "", "採取方法": "", "診断": "", "分化度": "", '
    '"病期": "", "腫瘍径": "", "浸潤範囲": "", "断端": "", "その他所見": "" }}'
)

INSTR_TEMPLATE = (
    "#指示\n"
    "次の文章を以下のJSON形式に構造化してください。\n"
    f"{SCHEMA_STR}\n\n"
    "#抽出対象\n"
    "{}"
)

# === ④ answer_ja のJSON整形補助 ===
def _to_json_str(s):
    if s is None or (isinstance(s, float) and math.isnan(s)):
        return "{}"
    if isinstance(s, dict):
        return json.dumps(s, ensure_ascii=False)
    if isinstance(s, str):
        try:
            repaired = s.replace('""', '"')
            parsed = json.loads(repaired)
            return json.dumps(parsed, ensure_ascii=False)
        except Exception:
            return s
    return str(s)

# === ⑤ Hugging Face Dataset → SFT形式変換 ===
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    q_ja_list = examples.get("question_ja", [])
    a_ja_list = examples.get("answer_ja", [])

    texts = []
    for q_ja, a_ja in zip(q_ja_list, a_ja_list):
        q_ja = "" if q_ja is None or (isinstance(q_ja, float) and math.isnan(q_ja)) else str(q_ja).strip()
        instr = INSTR_TEMPLATE.format(q_ja)
        out_json_str = _to_json_str(a_ja).strip()
        text = instr + "\n" + out_json_str + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

dataset = raw_ds_train.map(formatting_prompts_func, batched=True, remove_columns=raw_ds_train.column_names)

print("✅ dataset ready:", dataset)
print(dataset[0]["text"])

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

✅ dataset ready: Dataset({
    features: ['text'],
    num_rows: 95
})
#指示
次の文章を以下のJSON形式に構造化してください。
{ "臓器": "", "採取方法": "", "診断": "", "分化度": "", "病期": "", "腫瘍径": "", "浸潤範囲": "", "断端": "", "その他所見": "" }

#抽出対象
受領日：臨床診断および既往歴：偶然発見の3 cm左上極腎腫瘍。提出標本：1：左上極腎臓；部分腎切除。診断：1．左上極腎臓；部分腎切除：腫瘍型：腎細胞癌－従来型（透明細胞型）。フールマン核グレード：核グレードII/IV。腫瘍サイズ：最大径は2.4 cm。局所浸潤（腎皮質型に対して）：識別されず。腎静脈浸潤：識別されず。手術縁：腫瘍非存在。非腫瘍性腎臓：局所的慢性炎症および局所的表層性糸球体硬化を示す。副腎：識別されず。リンパ節：識別されず。腎細胞癌／オノコチオーマのステージング：pT1 腫瘍は最大径7.0 cm以下で腎臓内に限定される。上記診断は、私のスライド（および／または他の材料）の個人的検査に基づき、かつ、本報告書を検討し承認したことを証明する。肉眼所見：1）標本は凍結切片診断のため新鮮に受領され、「左上極腎腫瘍・縫合痕・深部縁」とラベル付けされている。6.6 × 3.7 × 2.9 cmの楔形腎臓部分からなり、深部縁を示す縫合痕が付随し、周囲腎脂肪組織12.2 × 8.5 × 2.7 cmが付属している。同一容器内に別個に11.6 × 8.4 × 4.6 cmの特記すべきことのない線維脂肪組織の集塊が受領されている。縁は黒インクで染色され、標本は連続断面化され、明確な境界を有する皮質起源の黄色、局所的に出血性の2.4 × 2.0 × 1.9 cmの腫瘍が確認された。腫瘍は肉眼的に腎実質内に限定されており、周囲腎脂肪への延展は認められない。切除縁からの距離は1.1 cmである。最も近い縁の代表的断面を凍結切片診断のために提出した。代表的提出。写真を撮影した。腫瘍の一部をTPSのために提出した。断面の要約：FSC－凍結切片コントロール。T－腫瘍。M－縁。RS－代表的断面。断面の要約：第1部：左上極腎臓；部分腎切除。ブロック。断面。部位。PCs。1．fs

###学習時のパラメータ設定

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 2,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/95 [00:00<?, ? examples/s]

###メモリ使用量

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.609 GB of memory reserved.


###学習の実行

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 95 | Num Epochs = 3 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 16,908,288 of 1,884,522,496 (0.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,2.192000
10,1.979900
15,1.829400
20,1.748200
25,1.793500
30,1.710400


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

229.8088 seconds used for training.
3.83 minutes used for training.
Peak reserved memory = 3.494 GB.
Peak reserved memory for training = 1.885 GB.
Peak reserved memory % of max memory = 23.703 %.
Peak reserved memory for training % of max memory = 12.787 %.


In [ ]:
model.save_pretrained("llm-jp-3.1-1.8b_tcga_sft_n95_step30_lora")  # Local saving
tokenizer.save_pretrained("llm-jp-3.1-1.8b_tcga_sft_n95_step30_lora")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('llm-jp-3.1-1.8b_tcga_sft_n95_step30_lora/tokenizer_config.json',
 'llm-jp-3.1-1.8b_tcga_sft_n95_step30_lora/special_tokens_map.json',
 'llm-jp-3.1-1.8b_tcga_sft_n95_step30_lora/tokenizer.json')

In [ ]:
SCHEMA_STR = (
    '{ "臓器": "", "採取方法": "", "診断": "", "分化度": "", '
    '"病期": "", "腫瘍径": "", "浸潤範囲": "", "断端": "", "その他所見": "" }'
)

def build_prompt(text):
    return (
        "#指示\n"
        "次の文章を以下のJSON形式に構造化してください。\n"
        f"{SCHEMA_STR}\n\n"
        "#抽出対象\n"
        f"{text}"
    )

###学習後の出力確認

In [ ]:
GEN_KWARGS = dict(
    max_new_tokens=512,
    do_sample=True,
    top_p=0.95,
    temperature=0.7,
    repetition_penalty=1.05,
)

In [ ]:
import torch
from unsloth import FastLanguageModel

# ----------------------------------------------------
# ▼▼▼ ステップ1：初期設定（ループの前に一度だけ実行） ▼▼▼
# ----------------------------------------------------
print("モデルを推論用に最適化します...（この処理は一度だけ実行されます）")

# Unslothによる推論最適化
FastLanguageModel.for_inference(model)
# 評価モードに設定
model.eval()

# PADトークンの設定
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

モデルを推論用に最適化します...（この処理は一度だけ実行されます）


In [ ]:
# ----------------------------------------------------
# ▼▼▼ ステップ2：推論ループと結果の保存 ▼▼▼
# ----------------------------------------------------
results = [] # 推論結果を格納するための空のリストを初期化

# raw_ds_eval のすべてのサンプルに対してループを実行
for idx, sample in enumerate(raw_ds_eval):
    print(f"==================== サンプル {idx} ====================")

    # 1. データセットから元の質問文を取得し、プロンプトを構築
    # ★ JSON保存用に元の質問文も変数に保持しておく
    original_question = sample["question_ja"]
    text = build_prompt(original_question)

    # print("\n--- 入力プロンプト ---")
    # print(text)

    # 2. プロンプトをトークナイズ
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)

    # 3. 推論の実行
    with torch.no_grad():
        output = model.generate(
            tokenized_input,
            **GEN_KWARGS,
        )[0]

    # 4. 生成された部分だけをデコード
    input_token_len = len(tokenized_input[0])
    generated_tokens = output[input_token_len:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    # 5. 生成結果をコンソールに表示
    print("\n--- モデルの生成結果 (生成部分のみ) ---")
    print(generated_text.strip()) # .strip()で前後の空白を除去
    print(f"==================== サンプル {idx} 終了 ====================\n")

    # 6. 結果をリストに追加
    results.append({
        "input": original_question,           # 元の質問文
        "model_output": generated_text.strip() # モデルの出力
    })

    # #【任意】テスト用に最初の数件でループを止めたい場合
    # if idx >= 1:
    #     break

==================== サンプル 0 ====================


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- モデルの生成結果 (生成部分のみ) ---
{"臓器": "腫瘍", "採取方法": "手術摘出", "診断": "中程度の悪性度（III度）", "分化度": "低悪性度（WHO病理学分類II度）", "病期": "", "腫瘍径": "", "浸潤範囲": "", "断端": "", "その他所見": "散在するMIB-1反応性細胞が認められる。"}
==================== サンプル 0 終了 ====================

==================== サンプル 1 ====================

--- モデルの生成結果 (生成部分のみ) ---
{"臓器": "精巣", "採取方法": "精巣内精巣胚芽腫", "診断": "精巣胚芽腫", "分化度": "未分化", "病期": "詳細不明", "腫瘍径": "6.5 cm", "浸潤範囲": "精巣網への侵襲あり", "断端": "精巣網への侵襲あり", "その他所見": "血管内侵襲あり"}
==================== サンプル 1 終了 ====================

==================== サンプル 2 ====================

--- モデルの生成結果 (生成部分のみ) ---
組織学的診断：A. 腫瘍陰性（0/20）。B. 膀胱前立腺：10% pT3a、0% pN、0% pM、0% pM+、0% pT3b、0% pT3M、0% pT3M+、0% pT3M++、0% pT3M+++。右側尿管：3% pT4a、0% pT4b、0% pT4c、0% pT4d、0% pT4e、0% pT4f、0% pT4g、0% pT4h、0% pT4i、0% pT4j、0% pT4k、0% pT4l、0% pT4m、0% pT4n、0% pT4o、0% pT4p、0% pT4q、0% pT4r、0% pT4s、0% pT4t、0% pT4u、0% pT4v、0% pT4w、0% pT4x、0% pT4y、0% pT4z。左側尿管：1% pT5a、0% pT5b、0% pT5c、0% pT5d、0% pT5e、0% pT5f、0% pT5g、0% pT5h、0% pT5i、0% pT5j、0% 

Unsloth: Input IDs of shape torch.Size([1, 2567]) with length 2567 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.



--- モデルの生成結果 (生成部分のみ) ---
A: { "臓器": "乳腺", "採取方法": "マステクトミー", "診断": "浸潤性乳管癌", "分化度": "未確定", "病期": "IIIb", "腫瘍径": "2.2 x 1.5 x 5 cm", "浸潤範囲": "乳房全領域", "断端": "乳房全領域", "その他所見": "リンパ節への浸潤陽性"}
==================== サンプル 3 終了 ====================

==================== サンプル 4 ====================

--- モデルの生成結果 (生成部分のみ) ---
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [ ]:
# ----------------------------------------------------
# ▼▼▼ ステップ3：JSONファイルへの書き込み ▼▼▼
# ----------------------------------------------------
output_filename_sft = f"eval_result_sft.json"

# --- JSONファイルに保存 ---
with open(output_filename_sft , "w", encoding="utf-8") as f:
    # ensure_ascii=False で日本語が文字化けしないようにする
    # indent=2 で人間が読みやすいように整形する
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"✅ 全ての推論が完了し、結果を {output_filename_sft } に保存しました。")

✅ 全ての推論が完了し、結果を eval_result_sft.json に保存しました。


In [ ]:
import pandas as pd

# --- 推論結果を DataFrame に変換 ---
df_results = pd.DataFrame(results)

# --- 表示設定 ---
pd.set_option('display.max_colwidth', None)   # 長文を省略せず表示
pd.set_option('display.max_rows', 100)        # 表示行数を調整（必要なら増やす）
pd.set_option('display.width', None)          # 横幅を自動調整

# --- Colab上でスクロール付きHTML表示 ---
from IPython.display import display, HTML

# 高さ・幅を指定してスクロール可能に
scrollable_html = df_results.to_html(escape=False)
display(HTML(f'<div style="max-height:600px; overflow-y:auto; border:1px solid #ccc;">{scrollable_html}</div>'))

,input,model_output
0,病理報告書（追加説明）：散在するMIB-1反応性細胞が認められる。最も増殖活性の高い領域において、ラベリングインデックスは5.0％と算出される。これは中程度の増殖活性を示しており、一般的に低悪性度の組織学的所見と整合する。 \n追加診断：オリゴアストロサイトーマ、オリゴデンドログリオーマ優勢。低悪性度（WHO病理学分類II度）。MIB-1ラベリングインデックス：5.0％。 \n顕微鏡的説明：切片は軽度から中程度の高細胞性を示す膠質腫瘍を示す。主な細胞群はマイクロジミストサイトである。細胞の少数のみが核周囲にハローを示す。また、核がやや長楕円形で、やや好酸性の細胞質を示す少数の細胞群も認められる。有糸分裂像は認められない。微小血管増殖や壊死は認められない。,"{""臓器"": ""腫瘍"", ""採取方法"": ""手術摘出"", ""診断"": ""中程度の悪性度（III度）"", ""分化度"": ""低悪性度（WHO病理学分類II度）"", ""病期"": """", ""腫瘍径"": """", ""浸潤範囲"": """", ""断端"": """", ""その他所見"": ""散在するMIB-1反応性細胞が認められる。""}"
1,病理学的要約報告。右側腹部精巣の摘出；精巣胚芽腫；径6.5 cm；可用スライドに存在する精巣内に腫瘍が限定；血管内侵襲あり；精巣網への侵襲あり。＝精巣摘出術施行日：病理医。,"{""臓器"": ""精巣"", ""採取方法"": ""精巣内精巣胚芽腫"", ""診断"": ""精巣胚芽腫"", ""分化度"": ""未分化"", ""病期"": ""詳細不明"", ""腫瘍径"": ""6.5 cm"", ""浸潤範囲"": ""精巣網への侵襲あり"", ""断端"": ""精巣網への侵襲あり"", ""その他所見"": ""血管内侵襲あり""}"
2,"性別：男性。人種：白人。報告日：最終病理診断。 \nA. 左側骨盤リンパ節；解剖：－20個のリンパ節、腫瘍なし（0/20）。 \nB. 右側骨盤リンパ節；解剖：－13個のリンパ節、腫瘍なし（0/13）。 \nC. 尿路膀胱、前立腺、精巣付属腺、精巣管；膀胱前立腺摘出術：－膀胱に浸潤性尿路上皮癌を伴い、局所的扁平上皮分化あり、病理学的パラメータを参照のこと。 \n－腫瘍は膀胱筋層に浸潤し、局所的顕微鏡的膀胱外軟部組織への拡散あり。 \n－前立腺尿道に局所的尿路上皮原位癌あり、前立腺管への拡散あり。 \n－尿道、尿管および軟部組織縁、腫瘍なし。 \nD. 右側遠位尿管；切除：－尿管の一部、腫瘍なし。 \nE. 双側精巣管；切除：－精巣管、腫瘍なし。 \n\n尿路上皮癌の病理学的パラメータ。 \n1. 腫瘍型：浸潤性尿路上皮癌、局所的扁平上皮分化あり（10%）。＜10% pen TSS 9/7/12.les。 \n2. 腫瘍のグレード：高グレード。 \n3. 浸潤深さ：顕微鏡的膀胱外軟部組織への拡散（スライド C8）。 \n4. 腫瘍の分布：単発；4.0 cm；右側側壁、前壁および後壁。 \n5. 尿管縁：腫瘍陰性。 \n6. 遠位尿道縁：腫瘍陰性。 \n7. 軟部組織縁または漿膜：腫瘍陰性。 \n8. リンパ節：腫瘍陰性（0/33）。 \n9. pTNM：pT3a,N0,MX。有効。チェックリストは、アメリカ共同腫瘍学会（AJCC）および国際がん連盟（UICC）の膀胱用第7版TNM分類システムを用いる。解釈は担当病理医により実施され、レジデント／フェローとレビューされた。 \n\n臨床歴：高グレード尿路上皮癌を有する60歳の男性が、膀胱前立腺摘出術およびイレオコンダクト術を施行中。 \n\n受領標本： \nA：左側骨盤リンパ節 \nB：右側骨盤リンパ節 \nC：膀胱、前立腺、精巣付属腺、精巣管 \nD：右側遠位尿管 \nE：双側精巣管 \n\n肉眼所見：標本は5つの容器に分けて受領され、それぞれ患者名および医療記録番号でラベル付けされている。 \nA. 最初の容器はさらに「左側骨盤リンパ節」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、淡黄色の組織塊で、6.0 x 4.5 x 2.0 cmの大きさ。22個のリンパ節が確認され、最大径は0.2～2.4 cmの範囲。以下の通り提出された： \nA1：最大のリンパ節を切断。 \nA2：1個のリンパ節を切断。 \nA3：1個のリンパ節を切断。 \nA4-5：各4個のリンパ節。 \nA6：3個のリンパ節。 \nA7-8：各4個のリンパ節。 \n\nB. 2番目の容器はさらに「右側骨盤リンパ節」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、淡黄色の組織塊で、10.5 x 8.5 x 2.0 cmの大きさ。15個のリンパ節が確認され、最大径は0.2～4.5 cmの範囲。以下の通り提出された： \nB1-2：最大のリンパ節。 \nB3：1個のリンパ節。 \nB4：1個のリンパ節を切断。 \nB5：1個のリンパ節。 \nB6-7：1個のリンパ節。 \nB8：3個のリンパ節。 \nB9：2個のリンパ節。 \nB10：5個のリンパ節。 \n\nC. 3番目の容器はさらに「膀胱、前立腺、精巣付属腺、精巣管」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、21 x 11 x 5.5 cmの膀胱前立腺摘出術標本で、9.5 x 8.5 x 5.0 cmの膀胱と付属脂肪組織、および4.2 x 3.8 x 3.0 cmの前立腺からなる。右側精巣付属腺は3.0 x 1.5 x 0.5 cm、右側精巣管は6.0 cm。左側精巣付属腺は2.8 x 1.5 x 0.6 cm、左側精巣管は6.5 cm。右側尿管 stump は長さ0.5 cm、直径0.9 cmまで拡張、左側は長さ0.6 cm、直径0.1 cm、両者とも完全で通気性のある腔を示す。右側前立腺は青インク、左側前立腺は黒インクで染色。膀胱は黒インクで染色。膀胱および前立腺は尿道に沿って前側に切開。前立腺は薄い、完全な膜様被膜に囲まれており、切断面ではゴム状、ピンク～淡黄色の尿道周囲の結節状変化を示し、離れた腫瘤や硬結は認めない。切開された膀胱は、右側側壁に4.0 x 3.5 x 1.0 cmの硬い、淡黄色の腫瘤を示し、後壁および前壁に拡がっている。切断面では、腫瘤は粘膜を貫通しており、深部縁から1.8 cmの位置にある。周囲の膀胱粘膜は浮腫、しわ状、ピンク～淡黄色で、壁厚は1.0～1.5 cm。三角部に隣接する両側尿管口が確認され、プローブで通気性あり。精巣付属脂肪組織内に2個のリンパ節候補が確認され、それぞれ0.4 cmおよび0.6 cm。代表的断面は以下の通り提出された： \nC1：遠位前立腺尿道縁（先端部）。 \nC2：右側尿管切除縁。 \nC3：左側尿管切除縁。 \nC4：前立腺先端部を連続的に切断。 \nC5-10：膀胱腫瘍、後側外側（C6-7、C9-10は連続）。 \nC11：膀胱頂部の腫瘍非関与粘膜。 \nC12：膀胱後壁の腫瘍非関与粘膜。 \nC13-14：膀胱腫瘍、前側外側。 \nC15：膀胱前壁の腫瘍非関与粘膜。 \nC16-18：先端部から基底部まで代表的右側前立腺。 \nC19-21：先端部から基底部まで代表的左側前立腺。 \nC22：2個のリンパ節候補。 \n\nD. 4番目の容器はさらに「右側遠位尿管」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、淡黄色の脂肪組織を伴う、方向不明の淡黄色組織片で、長さ2.5 cm、直径0.9 cm。標本はすべてD1-2に提出された。 \n\nE. 5番目の容器はさらに「双側精巣管」と識別されている。新鮮に受領され、ホルマリンに浸漬されたのは、ピンク～淡黄色の組織片2個で、精巣管に一致する。1つは長さ0.5 cm、直径0.4 cm。もう1つは長さ4.5 cm、直径0.4 cm。代表的断面はE1に提出された。",組織学的診断：A. 腫瘍陰性（0/20）。B. 膀胱前立腺：10% pT3a、0% pN、0% pM、0% pM+、0% pT3b、0% pT3M、0% pT3M+、0% pT3M++、0% pT3M+++。右側尿管：3% pT4a、0% pT4b、0% pT4c、0% pT4d、0% pT4e、0% pT4f、0% pT4g、0% pT4h、0% pT4i、0% pT4j、0% pT4k、0% pT4l、0% pT4m、0% pT4n、0% pT4o、0% pT4p、0% pT4q、0% pT4r、0% pT4s、0% pT4t、0% pT4u、0% pT4v、0% pT4w、0% pT4x、0% pT4y、0% pT4z。左側尿管：1% pT5a、0% pT5b、0% pT5c、0% pT5d、0% pT5e、0% pT5f、0% pT5g、0% pT5h、0% pT5i、0% pT5j、0% pT5k、0% pT5l、0% pT5m、0% pT5n、0% pT5o、0% pT5p、0% pT5q、0% pT5r、0% pT5s、0% pT5t、0% pT5u、0% pT5v、0% pT5w、0% pT5x、0% pT5y、0% pT5z。右側前立腺：10% pT6a、0% pT6b、0% pT6c、0% pT6d、0% pT6e、0% pT6f、0% pT6g、0%
3,乳腺組織チェックリスト。標本タイプ：マステクトミー。

###学習後のモデルのアップロード_使用しません(アップロード後にモデルを推論すると出力がおかしいため)

In [ ]:
# # Merge to 16bit
# model.save_pretrained_merged("llm-jp-3.1-1.8b_tcga_sft_n100_step30", tokenizer, save_method = "merged_16bit",)
# model.push_to_hub_merged("morizon/llm-jp-3.1-1.8b_tcga_sft_n100_step30", tokenizer, save_method = "merged_16bit",token=HF_TOKEN)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `llm-jp-3.1-1.8b_tcga_sft_n100_step30`: 100%|██████████| 1/1 [00:46<00:00, 46.10s/it]


Successfully copied all 1 files from cache to `llm-jp-3.1-1.8b_tcga_sft_n100_step30`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [01:34<00:00, 94.36s/it]


Unsloth: Merge process complete. Saved to `/content/llm-jp-3.1-1.8b_tcga_sft_n100_step30`
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0001-of-00001.safetensors:   1%|1         | 41.9MB / 3.74GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [02:22<00:00, 142.03s/it]


Unsloth: Merge process complete. Saved to `/content/morizon/llm-jp-3.1-1.8b_tcga_sft_n100_step30`


In [ ]:
# # LoRAアダプタだけ保存
# model.save_pretrained_merged("llm-jp-3.1-1.8b_tcga_sft_n100_step30_lora", tokenizer, save_method = "lora",)
# model.push_to_hub_merged(
#     "morizon/llm-jp-3.1-1.8b_tcga_sft_n100_step30_lora",
#     tokenizer=tokenizer,
#     save_method="lora",
#     token=HF_TOKEN
# )

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `llm-jp-3.1-1.8b_tcga_sft_n100_step30_lora`: 100%|██████████| 1/1 [01:14<00:00, 74.82s/it]


Successfully copied all 1 files from cache to `llm-jp-3.1-1.8b_tcga_sft_n100_step30_lora`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [01:44<00:00, 104.56s/it]


Unsloth: Merge process complete. Saved to `/content/llm-jp-3.1-1.8b_tcga_sft_n100_step30_lora`
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0001-of-00001.safetensors:   1%|          | 33.5MB / 3.74GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [02:28<00:00, 148.52s/it]


Unsloth: Merge process complete. Saved to `/content/morizon/llm-jp-3.1-1.8b_tcga_sft_n100_step30_lora`


#03_eval

##03-1_学習前のモデルの評価_llm-jp/llm-jp-3.1-1.8b

In [ ]:
inference_model_1="llm-jp/llm-jp-3.1-1.8b"

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
import json

# --- 1. 準備 ---

# APIキーの設定 (必ずYOUR_API_KEYの部分を実際のキーに置き換えてください)
genai.configure(api_key=GOOGLE_API_KEY)
judge_model = genai.GenerativeModel('gemini-2.5-flash')

In [ ]:
print("--- Gemini API 接続テスト開始 ---")

# 簡単で無害なプロンプトを用意
test_prompt = "こんにちは！LLMについて簡潔に100文字程度で教えてください。"

try:
    print("APIにリクエストを送信中... (タイムアウト: 60秒)")

    # APIを呼び出す際に、request_optionsでタイムアウトを指定
    # ★★★ ここが重要 ★★★
    response = judge_model.generate_content(
        test_prompt,
        request_options={"timeout": 60}  # 60秒でタイムアウト
    )

    print("APIから応答がありました！")
    print("-" * 20)
    print("応答テキスト:")
    print(response.text)
    print("-" * 20)
    print("✅ テスト成功: APIは正常に動作しています。")

# google.api_core.exceptions.DeadlineExceeded のようなタイムアウト固有のエラーをキャッチすることも可能
except Exception as e:
    print("\n❌ テスト失敗: API呼び出し中にエラーが発生しました。")
    print("--- エラー詳細 ---")
    # タイムアウトした場合、エラーメッセージに "Deadline Exceeded" や "Timeout" といった文言が含まれます
    print(e)
    print("--------------------")
    # タイムアウトしたかどうかを判定する
    if "deadline" in str(e).lower() or "timeout" in str(e).lower():
        print("💡 ヒント: 60秒以内に応答がなかったため、タイムアウトした可能性があります。")


print("\n--- Gemini API 接続テスト終了 ---")

--- Gemini API 接続テスト開始 ---
APIにリクエストを送信中... (タイムアウト: 60秒)
APIから応答がありました！
--------------------
応答テキスト:
LLM（大規模言語モデル）は、大量のテキストデータを学習したAIです。文章生成、要約、翻訳、質問応答など、人間に近い自然な言葉で多様なタスクをこなします。AIとの対話を可能にする重要な技術です。
--------------------
✅ テスト成功: APIは正常に動作しています。

--- Gemini API 接続テスト終了 ---


In [ ]:
import json
import re
import pandas as pd
from datetime import datetime

# --- モデル名・ファイル名設定 ---
modelname = inference_model_1.split("/")[-1]
date_str = datetime.now().strftime("%Y%m%d-%H%M%S")
input_filename = f"eval_result_{modelname}.json"
output_filename = f"eval_scored_results_{modelname}_{date_str}.json"
csv_filename = f"eval_scored_results_{modelname}_{date_str}.csv"

# --- 推論結果をロード ---
with open(input_filename, "r", encoding="utf-8") as f:
    eval_results = json.load(f)

evaluated = []

# --- Geminiで各出力を評価 ---
for item in eval_results:
    prompt = f"""
あなたは、医療テキスト構造化モデルの出力を評価する専門家です。

# 入力テキスト
{item["input"]}

# モデルの出力
{item["model_output"]}

次の3つの観点から評価してください。
1. **形式の正確性**：出力が構造化形式(JSON)として妥当か。
2. **情報の網羅性**：必要な情報（臓器・診断・病期など）を十分に含んでいるか。
3. **内容の正確性**：抽出内容が入力テキストの事実と一致しているか。

---

評価結果は以下のフォーマットで、日本語で簡潔に出力してください。

score: <0.0〜1.0の数値>
形式の正確性: <50文字程度の説明>
情報の網羅性: <50文字程度の説明>
内容の正確性: <50文字程度の説明>
総合コメント: <全体的な評価を一文でまとめる>
"""

    judge_response = judge_model.generate_content(prompt)
    raw_text = judge_response.text.strip()

    # --- コードブロックを除去（```json ... ``` や ``` ... ```）---
    cleaned_text = re.sub(r"^```(?:json)?|```$", "", raw_text, flags=re.MULTILINE).strip()

    # --- JSONパース試行 ---
    try:
        evaluation = json.loads(cleaned_text)
    except json.JSONDecodeError:
        # JSONとしてパースできない場合、スコアだけ正規表現で抽出
        score_match = re.search(r'"?score"?\s*[:：]\s*([0-9.]+)', cleaned_text)
        score = float(score_match.group(1)) if score_match else None
        evaluation = {"score": score, "reason": cleaned_text}

    evaluated.append({
        "input": item["input"][:100] + "...",
        "output": item["model_output"][:150] + "...",
        "score": evaluation.get("score"),
        "reason": evaluation.get("reason"),
    })

    # # 最初の1件の処理が終わったので、ループを強制的に終了します。
    # print("テストのため、最初の1件で処理を停止します。")
    # break
    # -----------------------------------------------

# --- DataFrameにして保存 ---
df = pd.DataFrame(evaluated)
df.to_json(output_filename, orient="records", force_ascii=False, indent=2)
df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

print(f"✅ 採点結果を {csv_filename} と {output_filename} に保存しました。")

# --- Colabでスクロールしやすく全文表示 ---
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
display(df.head(10))

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1213.76ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2742.64ms


✅ 採点結果を eval_scored_results_llm-jp-3.1-1.8b_20251113-065952.csv と eval_scored_results_llm-jp-3.1-1.8b_20251113-065952.json に保存しました。


,input,output,score,reason
0,病理報告書（追加説明）：散在するMIB-1反応性細胞が認められる。最も増殖活性の高い領域において、ラベリングインデックスは5.0％と算出される。これは中程度の増殖活性を示しており、一般的に低悪性度の組...,"--- A: {\n ""臓器"": ""肝臓"",\n ""採取方法"": ""生検"",\n ""診断"": ""Mieb-1陽性肝細胞癌"",\n ""分化度"": ""低悪性度"",\n ""病期"": ""III度"",\n ""腫瘍径"": ""不明"",\n ""浸潤範囲"": ""不明"",\n ""断端"": ""不明"",\n ""その他所見""...",0.3,score: 0.3\n形式の正確性: JSON形式としては妥当であり、構文上の問題はない。\n情報の網羅性: 必要な項目は網羅しているが、MIB-1ラベリングインデックスなどの重要な数値が「その他所見」に分類されている。\n内容の正確性: 臓器、診断、病期（WHO分類II度をIII期と誤認）が入力テキストと全く異なり、致命的な誤りが多い。\n総合コメント: 形式は正しいものの、臓器や診断といった主要情報の抽出に重大な誤りが複数あり、実用には適さないレベルである。
1,病理学的要約報告。右側腹部精巣の摘出；精巣胚芽腫；径6.5 cm；可用スライドに存在する精巣内に腫瘍が限定；血管内侵襲あり；精巣網への侵襲あり。＝精巣摘出術施行日：病理医。...,＝精巣生検実施後。＝卵巣・子宮摘出術施行後。＝精巣癌確定診断。＝精巣癌ステージIV、リンパ節転移陽性、遠隔転移陰性、腹膜播種なし。＝精巣癌の切除と同時に、隣接部組織も全摘出。＝精巣切除時に精巣体側の精管は温存。＝精巣残存部分には精巣上体も存在。＝精巣残存部分から再発なし。＝精巣残存部分の一部に微小リ...,0.0,score: 0.0\n形式の正確性: JSON形式ではなく、単なるテキストの羅列であるため、構造化出力として不適切です。\n情報の網羅性: 入力テキストの主要な病理学的情報（腫瘍径、侵襲部位など）が欠落しており、根拠のない情報や無関係な情報が多数含まれています。\n内容の正確性: ステージング、転移、再発に関する情報など、入力テキストに存在しない誤った内容が多く、抽出も不正確です。\n総合コメント: 形式、網羅性、内容の正確性の全ての観点において著しく問題があり、モデルの出力として全く機能していません。
2,性別：男性。人種：白人。報告日：最終病理診断。 \nA. 左側骨盤リンパ節；解剖：－20個のリンパ節、腫瘍なし（0/20）。 \nB. 右側骨盤リンパ節；解剖：－13個のリンパ節、腫瘍なし（0/13）...,症例の病理学的解析： \n病理学者は、複数の解剖学的領域における腫瘍の発生部位を特定した。腫瘍の発生は、左骨盤に限局した領域で最も強く、他の部分にも存在していることが判明した。腫瘍の発生には、いくつかの病理学的特徴が見られた。 \n\n1. 細胞の異質性：腫瘍細胞は主に単一の形態に固執せず、多様な形態...,0.1,score: 0.1\n形式の正確性: 構造化形式（JSON）ではなく、箇条書きのテキスト形式で出力されており、不適切です。\n情報の網羅性: 重要な病期分類（pTNM）、具体的な腫瘍型、扁平上皮分化の割合、切除縁の状態など、多くの情報が欠落しています。\n内容の正確性: 腫瘍の発生部位、組織学的特徴、サイズ計測など、入力テキストと異なる誤った情報が多数含まれています。\n総合コメント: モデルは構造化形式の要件を満たさず、情報の欠落と誤りが非常に多く、専門家による評価には耐えられません。
3,乳腺組織チェックリスト。標本タイプ：マステクトミー。標本サイズ：指定なし。腫瘍部位：乳房。腫瘍サイズ：2.2 x 1.5 x 5 cm。肉眼的に明らかな病変：あり。組織学的型：浸潤性乳管癌。組織学的グ...,"{""臓器"":"""" , ""採取方法"":"""", ""診断"":"""", ""分化度"":"""", ""病期"":"""", ""腫瘍径"":"""", ""浸潤範囲"":"""", ""断端"":"""", ""その他所見"":""""} A: {\n ""臓器"": ""乳腺組織"",\n ""採取方法"": """",\n ""診断"": ""浸潤性乳管癌"",\n ""分化...",0.6,score: 0.6\n形式の正確性: モデル出力の`A:`より前の文字列が余計だが、主要な出力部分はJSON形式として妥当である。\n情報の網羅性: 採取方法、病期（リンパ節転移）、分化度（グレード）、断端、その他所見など、多くの重要情報が欠落している。\n内容の正確性: 抽出された「臓器」「診断」「腫瘍径」「浸潤範囲」の値は入力テキストの事実と正確に一致している。\n総合コメント: 形式に若干の不備があるが抽出内容は正確であり、最も重要な課題は情報の網羅性が著しく低いことである。
4,解釈および診断：1) 喉頭（部分喉頭切除術）。標本タイプ：部分喉頭切除術。腫瘍部位：喉頭、上喉頭部に拡延し、声帯部にまで及ぶ。腫瘍サイズ：最大径：3.6cm。組織学的タイプ：扁平上皮癌。組織学的グレー...,...,0.0,score: 0.0\n形式の正確性: モデルの出力が提供されていないため、評価できません。\n情報の網羅性: モデルの出力が提供されていないため、評価できません。\n内容の正確性: モデルの出力が提供されていないため、評価できません。\n総合コメント: モデルの出力が提供されていないため、評価を実行できませんでした。


##03-2_モデルの評価_llm-jp/llm-jp-3.1-1.8b-instruct4

In [ ]:
inference_model_2="llm-jp/llm-jp-3.1-1.8b-instruct4"

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
import json

# --- 1. 準備 ---

# APIキーの設定 (必ずYOUR_API_KEYの部分を実際のキーに置き換えてください)
genai.configure(api_key=GOOGLE_API_KEY)
judge_model = genai.GenerativeModel('gemini-2.5-flash')

In [ ]:
print("--- Gemini API 接続テスト開始 ---")

# 簡単で無害なプロンプトを用意
test_prompt = "こんにちは！LLMについて簡潔に100文字程度で教えてください。"

try:
    print("APIにリクエストを送信中... (タイムアウト: 60秒)")

    # APIを呼び出す際に、request_optionsでタイムアウトを指定
    # ★★★ ここが重要 ★★★
    response = judge_model.generate_content(
        test_prompt,
        request_options={"timeout": 60}  # 60秒でタイムアウト
    )

    print("APIから応答がありました！")
    print("-" * 20)
    print("応答テキスト:")
    print(response.text)
    print("-" * 20)
    print("✅ テスト成功: APIは正常に動作しています。")

# google.api_core.exceptions.DeadlineExceeded のようなタイムアウト固有のエラーをキャッチすることも可能
except Exception as e:
    print("\n❌ テスト失敗: API呼び出し中にエラーが発生しました。")
    print("--- エラー詳細 ---")
    # タイムアウトした場合、エラーメッセージに "Deadline Exceeded" や "Timeout" といった文言が含まれます
    print(e)
    print("--------------------")
    # タイムアウトしたかどうかを判定する
    if "deadline" in str(e).lower() or "timeout" in str(e).lower():
        print("💡 ヒント: 60秒以内に応答がなかったため、タイムアウトした可能性があります。")


print("\n--- Gemini API 接続テスト終了 ---")

--- Gemini API 接続テスト開始 ---
APIにリクエストを送信中... (タイムアウト: 60秒)
APIから応答がありました！
--------------------
応答テキスト:
LLM（大規模言語モデル）は、大量のテキストデータで学習し、人間のように自然な文章を理解・生成するAIです。質問応答、要約、翻訳、創作など多岐にわたる言語タスクをこなします。
--------------------
✅ テスト成功: APIは正常に動作しています。

--- Gemini API 接続テスト終了 ---


In [ ]:
import json
import re
import pandas as pd
from datetime import datetime

# --- モデル名・ファイル名設定 ---
modelname = inference_model_2.split("/")[-1]
date_str = datetime.now().strftime("%Y%m%d-%H%M%S")
input_filename = f"eval_result_{modelname}.json"
output_filename = f"eval_scored_results_{modelname}_{date_str}.json"
csv_filename = f"eval_scored_results_{modelname}_{date_str}.csv"

# --- 推論結果をロード ---
with open(input_filename, "r", encoding="utf-8") as f:
    eval_results = json.load(f)

evaluated = []

# --- Geminiで各出力を評価 ---
for item in eval_results:
    prompt = f"""
あなたは、医療テキスト構造化モデルの出力を評価する専門家です。

# 入力テキスト
{item["input"]}

# モデルの出力
{item["model_output"]}

次の3つの観点から評価してください。
1. **形式の正確性**：出力が構造化形式(JSON)として妥当か。
2. **情報の網羅性**：必要な情報（臓器・診断・病期など）を十分に含んでいるか。
3. **内容の正確性**：抽出内容が入力テキストの事実と一致しているか。

---

評価結果は以下のフォーマットで、日本語で簡潔に出力してください。

score: <0.0〜1.0の数値>
形式の正確性: <50文字程度の説明>
情報の網羅性: <50文字程度の説明>
内容の正確性: <50文字程度の説明>
総合コメント: <全体的な評価を一文でまとめる>
"""

    judge_response = judge_model.generate_content(prompt)
    raw_text = judge_response.text.strip()

    # --- コードブロックを除去（```json ... ``` や ``` ... ```）---
    cleaned_text = re.sub(r"^```(?:json)?|```$", "", raw_text, flags=re.MULTILINE).strip()

    # --- JSONパース試行 ---
    try:
        evaluation = json.loads(cleaned_text)
    except json.JSONDecodeError:
        # JSONとしてパースできない場合、スコアだけ正規表現で抽出
        score_match = re.search(r'"?score"?\s*[:：]\s*([0-9.]+)', cleaned_text)
        score = float(score_match.group(1)) if score_match else None
        evaluation = {"score": score, "reason": cleaned_text}

    evaluated.append({
        "input": item["input"][:100] + "...",
        "output": item["model_output"][:150] + "...",
        "score": evaluation.get("score"),
        "reason": evaluation.get("reason"),
    })

    # # 最初の1件の処理が終わったので、ループを強制的に終了します。
    # print("テストのため、最初の1件で処理を停止します。")
    # break
    # -----------------------------------------------

# --- DataFrameにして保存 ---
df = pd.DataFrame(evaluated)
df.to_json(output_filename, orient="records", force_ascii=False, indent=2)
df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

print(f"✅ 採点結果を {csv_filename} と {output_filename} に保存しました。")

# --- Colabでスクロールしやすく全文表示 ---
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
display(df.head(10))

✅ 採点結果を eval_scored_results_llm-jp-3.1-1.8b-instruct4_20251113-065113.csv と eval_scored_results_llm-jp-3.1-1.8b-instruct4_20251113-065113.json に保存しました。


,input,output,score,reason
0,病理報告書（追加説明）：散在するMIB-1反応性細胞が認められる。最も増殖活性の高い領域において、ラベリングインデックスは5.0％と算出される。これは中程度の増殖活性を示しており、一般的に低悪性度の組...,"```json\n{\n ""臓器"": ""不明"", // 文章内では具体的な臓器名は記載されていません\n ""採取方法"": ""不明"", // 文章内では具体的な採取方法は記載されていません\n ""診断"": {\n ""主要所見"": ""散在するMIB-1反応性細胞"",\n ""追加所見"": ""オリゴア...",0.80,score: 0.8\n形式の正確性: JSONコメントが含まれており、厳密なJSON形式としては無効です。\n情報の網羅性: MIB-1ラベリングインデックス5.0%という数値情報が専用のキーとして抽出されていません。\n内容の正確性: 抽出された全ての情報は入力テキストの事実と正確に一致しています。\n総合コメント: 抽出された情報の正確性は非常に高いものの、JSON形式の厳密な準拠と一部の数値情報の網羅性において改善の余地があります。
1,病理学的要約報告。右側腹部精巣の摘出；精巣胚芽腫；径6.5 cm；可用スライドに存在する精巣内に腫瘍が限定；血管内侵襲あり；精巣網への侵襲あり。＝精巣摘出術施行日：病理医。...,"以下のJSON形式で、指定された情報を構造化しました。\n\n```json\n{\n ""臓器"": ""精巣"",\n ""採取方法"": ""精巣胚芽腫のため、精巣内での摘出術"",\n ""診断"": ""精巣胚芽腫"",\n ""分化度"": ""詳細な分化度は記載なし"",\n ""病期"": ""精巣胚芽腫"",\n ""腫瘍径"":...",0.70,score: 0.7\n形式の正確性: JSON形式として適切に構造化されており、形式上の問題はありません。\n情報の網羅性: 「病期」の情報が診断名になっており不正確。「浸潤範囲」に重要な情報「精巣内に腫瘍が限定」が欠落しています。\n内容の正確性: 「病期」が診断名と誤って解釈されています。「採取方法」の記述もやや不自然です。\n総合コメント: JSON形式は正確ですが、病期や浸潤範囲の解釈に誤りや情報不足が見られ、改善が必要です。
2,性別：男性。人種：白人。報告日：最終病理診断。 \nA. 左側骨盤リンパ節；解剖：－20個のリンパ節、腫瘍なし（0/20）。 \nB. 右側骨盤リンパ節；解剖：－13個のリンパ節、腫瘍なし（0/13）...,"```json\n{\n ""臓器"": ""尿路上皮"",\n ""採取方法"": ""生検"",\n ""診断"": ""浸潤性尿路上皮癌"",\n ""分化度"": ""高度に扁平上皮分化あり、10% pen TSS 9/7/12.les。"",\n ""病期"": ""T3aN0MX、pT3aN0MX"",\n ""腫瘍径"": {\n ...",0.20,score: 0.2\n形式の正確性: JSONとして文法エラー（閉じ括弧の欠損、値のカンマ）があり、パース不能なため、形式の正確性は低い。\n情報の網羅性: 採取方法が誤っており、臓器情報が不十分。病理学的パラメータの主要な情報（浸潤深さ、縁の状態など）が多く欠落している。\n内容の正確性: 採取方法、臓器の指定が誤り。「腫瘍径」のほとんどの情報が、リンパ節の数や臓器のサイズと混同されており、入力テキストの事実と一致しない。\n総合コメント: JSON形式として破綻しており、抽出された情報の大部分が不正確であるため、モデルの出力としては実用性に欠ける。
3,乳腺組織チェックリスト。標本タイプ：マステクトミー。標本サイズ：指定なし。腫瘍部位：乳房。腫瘍サイズ：2.2 x 1.5 x 5 cm。肉眼的に明らかな病変：あり。組織学的型：浸潤性乳管癌。組織学的グ...,"```json\n{\n ""臓器"": ""乳腺"",\n ""採取方法"": ""マステクトミー"",\n ""診断"": ""浸潤性乳管癌"",\n ""分化度"": ""指定なし"",\n ""病期"": ""ステージIV"",\n ""腫瘍径"": {\n ""長さ"": ""2.2 cm"",\n ""幅"": ""1.5 cm"",\n ...",0.67,score: 0.67\n形式の正確性: JSON形式として妥当であり、データの構造も適切に組まれている。\n情報の網羅性: 主要な情報は概ね含まれるが、ノッティンガムスコアの詳細項目やその他所見の一部に漏れがある。\n内容の正確性: 「病期」と「浸潤範囲」の抽出内容が入力テキストの事実と矛盾しており、特に診断に関わる重要な情報の誤りが複数存在する。\n総合コメント: 形式は正確であるが、診断に関わる重要な情報に誤りがあり、内容の正確性に大きな問題がある。
4,解釈および診断：1) 喉頭（部分喉頭切除術）。標本タイプ：部分喉頭切除術。腫瘍部位：喉頭、上喉頭部に拡延し、声帯部にまで及ぶ。腫瘍サイズ：最大径：3.6cm。組織学的タイプ：扁平上皮癌。組織学的グレー...,"以下は、提供された情報をJSON形式で整理したものです。\n\n```json\n{\n ""臓器"": ""喉頭"",\n ""採取方法"": ""部分喉頭切除術"",\n ""診断"": {\n ""1a-1"": ""扁平上皮癌"",\n ""1b-3"": ""中等度に分化"",\n ""1c-2"": ""原発腫瘍"",\n ...",0.10,score: 0.1\n形式の正確性: JSON形式で記述されているが、途中で切れており、妥当なJSONではない。\n情報の網羅性: 重要な病期分類（TNMの詳細）、リンパ節転移の個数、リンパ節外拡散、縁の状態、血管・神経周囲侵襲などの情報が大幅に欠落している。\n内容の正確性: 「分化度」の「高分化」や「病期」の抽象的な表現は入力テキストの事実と異なり、不正確な情報が多い。\n総合コメント: JSON形式が不完全である上、抽出された情報の網羅性が低く、内容の正確性にも大きな問題がある。


##03-3_学習後のモデルの評価

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
import json

# --- 1. 準備 ---

# APIキーの設定 (必ずYOUR_API_KEYの部分を実際のキーに置き換えてください)
genai.configure(api_key=GOOGLE_API_KEY)
judge_model = genai.GenerativeModel('gemini-2.5-flash')

In [ ]:
print("--- Gemini API 接続テスト開始 ---")

# 簡単で無害なプロンプトを用意
test_prompt = "こんにちは！LLMについて簡潔に100文字程度で教えてください。"

try:
    print("APIにリクエストを送信中... (タイムアウト: 60秒)")

    # APIを呼び出す際に、request_optionsでタイムアウトを指定
    # ★★★ ここが重要 ★★★
    response = judge_model.generate_content(
        test_prompt,
        request_options={"timeout": 60}  # 60秒でタイムアウト
    )

    print("APIから応答がありました！")
    print("-" * 20)
    print("応答テキスト:")
    print(response.text)
    print("-" * 20)
    print("✅ テスト成功: APIは正常に動作しています。")

# google.api_core.exceptions.DeadlineExceeded のようなタイムアウト固有のエラーをキャッチすることも可能
except Exception as e:
    print("\n❌ テスト失敗: API呼び出し中にエラーが発生しました。")
    print("--- エラー詳細 ---")
    # タイムアウトした場合、エラーメッセージに "Deadline Exceeded" や "Timeout" といった文言が含まれます
    print(e)
    print("--------------------")
    # タイムアウトしたかどうかを判定する
    if "deadline" in str(e).lower() or "timeout" in str(e).lower():
        print("💡 ヒント: 60秒以内に応答がなかったため、タイムアウトした可能性があります。")


print("\n--- Gemini API 接続テスト終了 ---")

--- Gemini API 接続テスト開始 ---
APIにリクエストを送信中... (タイムアウト: 60秒)


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 4920.82ms


APIから応答がありました！
--------------------
応答テキスト:
LLM（大規模言語モデル）は、大量のテキストを学習し、人間のように自然な言葉を理解・生成するAIです。会話、翻訳、要約、文章作成など多様なタスクをこなします。
--------------------
✅ テスト成功: APIは正常に動作しています。

--- Gemini API 接続テスト終了 ---


In [ ]:
import json
import re
import pandas as pd
from datetime import datetime

# --- モデル名・ファイル名設定 ---
date_str = datetime.now().strftime("%Y%m%d-%H%M%S")
input_filename_sft = output_filename_sft
output_filename_sft = f"eval_scored_results_{date_str}.json"
csv_filename_sft = f"eval_scored_results_{date_str}.csv"

# --- 推論結果をロード ---
with open(input_filename_sft, "r", encoding="utf-8") as f:
    eval_results = json.load(f)

evaluated_sft = []

# --- Geminiで各出力を評価 ---
for item in eval_results:
    prompt = f"""
あなたは、医療テキスト構造化モデルの出力を評価する専門家です。

# 入力テキスト
{item["input"]}

# モデルの出力
{item["model_output"]}

次の3つの観点から評価してください。
1. **形式の正確性**：出力が構造化形式(JSON)として妥当か。
2. **情報の網羅性**：必要な情報（臓器・診断・病期など）を十分に含んでいるか。
3. **内容の正確性**：抽出内容が入力テキストの事実と一致しているか。

---

評価結果は以下のフォーマットで、日本語で簡潔に出力してください。

score: <0.0〜1.0の数値>
形式の正確性: <50文字程度の説明>
情報の網羅性: <50文字程度の説明>
内容の正確性: <50文字程度の説明>
総合コメント: <全体的な評価を一文でまとめる>
"""


    judge_response = judge_model.generate_content(prompt)
    raw_text = judge_response.text.strip()

    # --- コードブロックを除去（```json ... ``` や ``` ... ```）---
    cleaned_text = re.sub(r"^```(?:json)?|```$", "", raw_text, flags=re.MULTILINE).strip()

    # --- JSONパース試行 ---
    try:
        evaluation = json.loads(cleaned_text)
    except json.JSONDecodeError:
        # JSONとしてパースできない場合、スコアだけ正規表現で抽出
        score_match = re.search(r'"?score"?\s*[:：]\s*([0-9.]+)', cleaned_text)
        score = float(score_match.group(1)) if score_match else None
        evaluation = {"score": score, "reason": cleaned_text}

    evaluated_sft.append({
        "input": item["input"][:100] + "...",
        "output": item["model_output"][:150] + "...",
        "score": evaluation.get("score"),
        "reason": evaluation.get("reason"),
    })

# --- DataFrameにして保存 ---
df_sft = pd.DataFrame(evaluated_sft)
df_sft.to_json(output_filename_sft, orient="records", force_ascii=False, indent=2)
df_sft.to_csv(csv_filename_sft, index=False, encoding="utf-8-sig")

print(f"✅ 採点結果を {csv_filename_sft} と {output_filename_sft} に保存しました。")

# --- Colabでスクロールしやすく全文表示 ---
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
display(df_sft.head(10))

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 6169.50ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2757.59ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1158.38ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 9461.92ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 4662.02ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1531.22ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 7068.12ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encodin

KeyboardInterrupt: 